In [ ]:
# %pip install azure-cognitiveservices-vision-customvision
# %pip install pylabel

In [9]:
#Import Azure cognitive services libraries 
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import pandas as pd

#Import other libraries used in this notebook 
import os, zipfile
from pathlib import PurePath
from os.path import exists
from decimal import *

from pylabel import importer
from tqdm import tqdm

In [34]:
ENDPOINT = ""
training_key = ""
PROJECT_ID = ""

prediction_key = ""
prediction_resource_id = ""

In [3]:
#Initialize objects used by Azure Congitive vision
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
# predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [5]:
#Create a new project
# publish_iteration_name = "detectModel"

obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")
project = trainer.create_project("test_project", domain_id=obj_detection_domain.id)

#If you browse to https://www.customvision.ai/ you should see a new project called "PyLabel Sample Dataset"

In [4]:
#Import annotations as a PyLabel dataset
# dataset = importer.ImportYoloV5(path="./squirrelsandnuts_train/squirrelsandnuts_train/labels/train",
#         path_to_images="../../images/train", 
#         img_ext="jpeg",
#         cat_names=['Squirrel','Nut']
#     )
# dataset.df.head(3)

In [29]:
# PROJECT_ID = project.id

In [6]:
classes = ['burner', 'cooktop']

#Create a tag for each class and store then in a dict where the class name is the key
tags = {}
for class_name in classes:
    tag = trainer.create_tag(PROJECT_ID, class_name)
    tags[class_name] = tag

tags

{'burner': <azure.cognitiveservices.vision.customvision.training.models._models_py3.Tag at 0x1dbb8b9f400>,
 'cooktop': <azure.cognitiveservices.vision.customvision.training.models._models_py3.Tag at 0x1dbb8b9f820>}

In [35]:
tags = {tag.name: tag.id for tag in trainer.get_tags(PROJECT_ID)}
tags

{'burner': 'd2cbd074-7074-475f-959e-0af3b14a4ad9',
 'cooktop': '7cf158fc-8c6f-488b-8c01-aa7ce681cb59'}

In [43]:
dff_correct = pd.read_csv('./burner_images/correct_burner_detections.csv')
dff_correct

,FileName,ClaimReference,bboxes
0,4df319f3-81ef-47ba-ad9b-b2eddac35592.jpg,Test_yago_222,"[{'left': 0.117004454, 'top': 0.48649615, 'wid..."
1,b57039ec-c1c3-40bc-a831-caa7a522d012.jpg,9704362873,"[{'left': 0.05457288, 'top': 0.15867461, 'widt..."
2,b753e287-43fb-4bb1-9e5b-ddd27dc5abbf.jpg,9704323816,"[{'left': 0.30414692, 'top': 0.25469273, 'widt..."
3,b9404bfc-8225-477e-9264-239e97a52043.jpg,9704392763,"[{'left': 0.5341124, 'top': 0.41454566, 'width..."
4,comp_b5f5db95-9cfc-4184-b5d3-904535854ff9.jpg,9704326889,"[{'left': 0.5626941, 'top': 0.4200442, 'width'..."
...,...,...,...
384,cf86c6e9-273a-4055-9475-8b7b49c65565.jpg,9704420088,"[{'left': 0.5418597, 'top': 0.45997715, 'width..."
385,acdddf64-cf71-446f-8762-1b6c41ee05ae.jpg,9704309154,"[{'left': 0.17351826, 'top': 0.23820734, 'widt..."
386,d6209279-db64-4c61-b362-151eeb56a94a.jpg,9704371968,"[{'left': 0.2798521, 'top': 0.5171221, 'width'..."
387,42ee0bcf-08ea-4298-b87b-73229f9103a9.jpg,9704371049,"[{'left': 0.301617, 'top': 0.20707689, 'width'..."


In [45]:
# cnt = 0 
for j in tqdm(range(len(dff_correct['FileName']))):
    # cnt +=1
    img_filename = dff_correct.loc[j,'FileName']
    img_path = './burner_images/correct_imgs/'+img_filename

    img_path = str(PurePath(img_path))
    assert exists(img_path), f"File does not exist: {img_path}"

    regions = []
    
    bboxes = eval(dff_correct.loc[j,'bboxes'])

    for i in bboxes:

        #Normalize the boundings box coordinates between 0 and 1
        x = i['left']
        y = i['top']
        w = i['width']
        h = i['height']
        label = i['label']

        regions.append(Region(
                    # tag_id=tags[label].id, 
                    tag_id=tags[label],
                    left=x,
                    top=y,
                    width=w,
                    height=h
                )
            )

    #Create an object with the image and all of the annotations for that image
    with open(img_path, mode="rb") as image_contents:
        image_and_annotations = [ImageFileCreateEntry(name=img_filename, contents=image_contents.read(), regions=regions)]

    
    #Upload the image and all annnotations for that image
    upload_result = trainer.create_images_from_files(PROJECT_ID, ImageFileCreateBatch(images=image_and_annotations))
    
    #If upload is not successful, print details about that image for debugging 
    if not upload_result.is_batch_successful:
        print("Image upload failed.")
        for image in upload_result.images:
            print(img_path)
            print("Image status: ", image.status)
            # print(regions)
            
    # if cnt==8:
    #     break

#This will take a few minutes 
print("Upload complete")

  0%|▎                                                                                                            | 1/389 [00:02<15:18,  2.37s/it]

Image upload failed.
burner_images\correct_imgs\4df319f3-81ef-47ba-ad9b-b2eddac35592.jpg
Image status:  OKDuplicate


  1%|▌                                                                                                            | 2/389 [00:04<14:30,  2.25s/it]

Image upload failed.
burner_images\correct_imgs\b57039ec-c1c3-40bc-a831-caa7a522d012.jpg
Image status:  OKDuplicate


  1%|▊                                                                                                            | 3/389 [00:05<12:04,  1.88s/it]

...

Image upload failed.
burner_images\correct_imgs\2457ad35-dc87-4601-83e8-5a19918cc667.jpg
Image status:  OKDuplicate


  2%|██▏                                                                                                          | 8/389 [00:25<28:33,  4.50s/it]

Image upload failed.
burner_images\correct_imgs\975620f4-7651-4946-b941-8a6c415f143b.jpg
Image status:  OKDuplicate


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 389/389 [16:39<00:00,  2.57s/it]

Upload complete
